In [9]:
import requests
import pandas as pd
import time

In [11]:

CSV_INPUT = "train_data.csv"
CSV_OUTPUT = "customer_feedback_analysis_test.csv"  # test output


In [12]:

def get_iam_token(api_key):
    resp = requests.post(
        "https://iam.cloud.ibm.com/identity/token",
        data={"apikey": api_key, "grant_type": "urn:ibm:params:oauth:grant-type:apikey"},
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    resp.raise_for_status()
    return resp.json()["access_token"]

In [ ]:
def process_customer_reviews(dataframe, auth_token):
    """Process customer reviews with batch saving every 20 iterations"""
    
    # API configuration
    api_endpoint = "https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
    request_headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {auth_token}"
    }
    
    # Initialize result columns
    for col in ["Sentiment", "Key_Issues", "Summary"]:
        dataframe[col] = ""
    
    # Process reviews in batches
    total_reviews = len(dataframe)
    batch_size = 20
    
    for batch_start in range(0, total_reviews, batch_size):
        batch_end = min(batch_start + batch_size, total_reviews)
        print(f"🔄 Processing batch {batch_start//batch_size + 1}: reviews {batch_start+1}-{batch_end}")
        
        # Process each review in current batch
        for current_idx in range(batch_start, batch_end):
            review_text = dataframe.iloc[current_idx]["Customer_Service"]
            
            # Generate analysis request
            analysis_results = _request_review_analysis(
                review_text, api_endpoint, request_headers, current_idx + 1, total_reviews
            )
            
            # Update dataframe with results
            _update_dataframe_row(dataframe, current_idx, analysis_results)
            
            # Rate limiting pause
            time.sleep(1)
        
        # Save progress after each batch
        _save_progress(dataframe, batch_end, total_reviews)

def _request_review_analysis(review_content, endpoint, headers, current_num, total_count):
    """Make API request for single review analysis"""
    
    analysis_prompt = _build_analysis_prompt(review_content)
    
    request_payload = {
        "input": analysis_prompt,
        "parameters": {"decoding_method": "greedy", "max_new_tokens": 200},
        "model_id": "ibm/granite-3-8b-instruct",
        "project_id": "xxxx",
        "moderations": {
            "hap": {"input": {"enabled": True}, "output": {"enabled": True}},
            "pii": {"input": {"enabled": True}, "output": {"enabled": True}},
            "granite_guardian": {"input": {"threshold": 1}}
        }
    }
    
    print(f"🟨 Processing review {current_num}/{total_count}...")
    
    try:
        response = requests.post(endpoint, headers=headers, json=request_payload)
        
        if response.status_code == 200:
            generated_output = response.json()["results"][0]["generated_text"]
            print(f"📝 Model response:\n{generated_output}\n")
            return _parse_analysis_response(generated_output, current_num - 1)
        else:
            print(f"🛑 Review {current_num}: API error {response.status_code} - {response.text}")
            return {"sentiment": "", "key_issues": "", "summary": ""}
            
    except Exception as e:
        print(f"🛑 Review {current_num}: Request failed - {str(e)}")
        return {"sentiment": "", "key_issues": "", "summary": ""}

def _build_analysis_prompt(review_text):
    """Construct the analysis prompt for the API"""
    return (
        f"Review: \"{review_text}\"\n\n"
        "Instructions:\n"
        "Analyze the above customer review and return the following:\n"
        "Sentiment: (Positive or Negative)\n"
        "Key_Phrases: (comma-separated list of key issues or highlights)\n"
        "Summary: Write a 1-2 sentence summary combining the sentiment and the key issues/highlights."
    )

def _parse_analysis_response(api_response, row_index):
    """Extract structured data from API response"""
    
    parsed_sentiment = ""
    parsed_key_phrases = ""
    parsed_summary = ""
    
    # Parse response line by line
    response_lines = api_response.splitlines()
    
    for response_line in response_lines:
        cleaned_line = response_line.strip()
        
        # Extract sentiment
        if cleaned_line.lower().startswith("sentiment:"):
            sentiment_value = cleaned_line.split(":", 1)[1].strip()
            if sentiment_value.lower() in ["positive", "negative"]:
                parsed_sentiment = sentiment_value
        
        # Extract key phrases
        elif cleaned_line.lower().startswith("key_phrases:"):
            parsed_key_phrases = cleaned_line.split(":", 1)[1].strip()
        
        # Extract summary
        elif cleaned_line.lower().startswith("summary:"):
            parsed_summary = cleaned_line.split(":", 1)[1].strip()
    
    # Validation check
    if not all([parsed_sentiment, parsed_key_phrases, parsed_summary]):
        print(f"⚠️ Incomplete analysis for row {row_index}. Full response:\n{api_response}\n")
    
    return {
        "sentiment": parsed_sentiment,
        "key_issues": parsed_key_phrases,
        "summary": parsed_summary
    }

def _update_dataframe_row(df, row_idx, analysis_data):
    """Update specific dataframe row with analysis results"""
    df.at[row_idx, "Sentiment"] = analysis_data["sentiment"]
    df.at[row_idx, "Key_Issues"] = analysis_data["key_issues"]
    df.at[row_idx, "Summary"] = analysis_data["summary"]

def _save_progress(df, processed_count, total_count):
    """Save current progress to output file"""
    df.to_csv(CSV_OUTPUT, index=False)
    print(f"💾 Progress saved: {processed_count}/{total_count} reviews completed\n")

def main():
    print("🔑 Getting IAM access token...")
    access_token = get_iam_token(API_KEY)
    print("✅ Token retrieved. Starting analysis...\n")
    
    # Load and process data
    review_dataframe = pd.read_csv(CSV_INPUT)
    process_customer_reviews(review_dataframe, access_token)
    
    print(f"\n✅ Analysis complete! Final output saved to: {CSV_OUTPUT}")

if __name__ == "__main__":
    main()


🔑 Getting IAM access token...
✅ Token retrieved. Starting analysis...

🔄 Processing batch 1: reviews 1-20
🟨 Processing review 1/200...
📝 Model response:


Sentiment: Negative
Key_Phrases: "not clean upon delivery"
Summary: The customer expressed disappointment with the vehicle's cleanliness upon delivery, indicating a negative sentiment towards the service.

🟨 Processing review 2/200...
📝 Model response:


Sentiment: Positive
Key_Phrases: Car pickup service, smooth experience, Enterprise
Summary: The customer had a positive experience with Enterprise, praising their car pickup service and the overall smoothness of the process.

Sentiment: Positive
Key_Phrases: Car pickup service, smooth experience, Enterprise
Summary: The customer had a positive experience with Enterprise, appreciating their car pickup service and the seamless nature of the transaction.

Sentiment: Positive
Key_Phrases: Car pickup service, smooth experience, Enterprise
Summary: The customer had a positive experience wi